In [1]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output 
from tqdm import tqdm
import pydensecrf.densecrf as dcrf
from pydensecrf.utils import unary_from_labels
import random

random_seed = 42
np.random.seed(random_seed)
random.seed(random_seed)

# RLE 디코딩 함수
def rle_decode(mask_rle, shape):
    if mask_rle == -1:
        return np.zeros(shape, dtype=np.uint8)
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# RLE 인코딩 함수
def rle_encode(mask):
    if np.sum(mask) == 0:
        return '-1'
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# 홀 채우기 함수 수정
def mask_filling(mask):
    # 마스크 내부 영역 추출
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filled_mask = np.zeros_like(mask)
    cv2.drawContours(filled_mask, contours, -1, 255, -1)
    return filled_mask

# CSV 파일 로드
csv_file = pd.read_csv('../submit/b5_cityscapes_040_42000steps.csv')  # CSV 파일 경로

# 결과를 저장할 새로운 DataFrame 생성
result_data = pd.DataFrame(columns=['img_id', 'mask_rle'])

# 각 사진과 마스크에 대해 예시 출력 및 결과 저장
for i in tqdm(range(60640)):
    # 사진 파일 경로 생성
    
    image_path = f'../data/test_img/{csv_file["img_id"][i]}.png'  # 이미지 파일 경로 생성 (확장자에 따라 수정해야 할 수 있습니다)

    # 사진 파일 확인
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"Image file not found: {image_path}")
    except Exception as e:
        print(str(e))
        assert False

    # 마스크 정보 로드 및 디코딩
    mask_data = csv_file['mask_rle'][i]
    mask = rle_decode(mask_data, image.shape[:2])

    # 작은 객체 제거
    min_object_size = 10  # 임의의 작은 객체 크기 기준 설정
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        area = cv2.contourArea(contour)
        if area < min_object_size:
            cv2.drawContours(mask, [contour], 0, 0, -1)

    # 마스크 내부 영역 채우기
    filled_mask = mask_filling(mask)

    # DenseCRF2D 입력 설정

    ############## 재욱이 추가 부분 ##################
    # unary = np.expand_dims(mask, axis=0)
    # unary = np.concatenate([unary, 1-unary], axis=0)
    # unary = unary.reshape((2, -1))
    # unary_float = unary.astype(np.float32)

    # # DenseCRF2D 객체 생성
    # d = dcrf.DenseCRF2D(224, 224, 2)  # width, height, nlabels

    # # unary potential 설정
    # d.setUnaryEnergy(unary_float)

    # d.addPairwiseGaussian(sxy=3, compat=3)

    # # DenseCRF2D 수행
    # Q = d.inference(10)  # 반복 횟수 설정

    # # 최적화된 마스크 가져오기
    # filled_mask = np.argmax(Q, axis=0).reshape((224, 224))


    # 결과를 DataFrame에 저장
    result_data.loc[i] = [csv_file['img_id'][i], rle_encode(filled_mask)]

# 결과 DataFrame을 final.csv 파일로 저장
result_data['mask_rle'] = result_data['mask_rle'].fillna(-1)
print(result_data.head)


100%|██████████| 60640/60640 [06:18<00:00, 160.09it/s]

<bound method NDFrame.head of            img_id                                           mask_rle
0      TEST_00000  19782 1 20003 8 20226 10 20257 7 20450 11 2048...
1      TEST_00001  35837 4 36059 6 36281 8 36504 9 36727 10 36949...
2      TEST_00002                                                 -1
3      TEST_00003  19 27 73 11 243 27 297 11 467 27 522 9 690 29 ...
4      TEST_00004  16891 21 16929 6 17114 24 17145 14 17337 47 17...
...           ...                                                ...
60635  TEST_60635                                                 -1
60636  TEST_60636  29094 4 29316 11 29538 14 29760 16 29979 22 30...
60637  TEST_60637  57 19 93 12 281 19 317 12 505 19 541 13 728 21...
60638  TEST_60638  14483 9 14700 19 14921 22 15144 24 15368 24 15...
60639  TEST_60639  11004 8 11226 12 11446 16 11666 20 11886 25 12...

[60640 rows x 2 columns]>


In [2]:
result_data.to_csv('../submit/b5_cityscapes_040_42000steps_yjypost.csv', index=False)

In [2]:
# CSV 파일 로드
import pandas as pd
result_data = pd.read_csv('../submit/deepexc_a15_post.csv')  # CSV 파일 경로
result_data['mask_rle'] = result_data['mask_rle'].fillna(-1)
print(result_data.head)
result_data.to_csv('../submit/deepexc_a15_post.csv', index=False)

<bound method NDFrame.head of            img_id                                           mask_rle
0      TEST_00000  20002 1 20226 9 20450 9 20480 8 20674 10 20703...
1      TEST_00001  35839 2 36060 5 36283 6 36506 7 36728 9 36951 ...
2      TEST_00002                                                 -1
3      TEST_00003  21 24 69 18 243 26 295 15 466 28 520 12 689 29...
4      TEST_00004  16682 2 16898 15 17115 24 17152 7 17338 46 175...
...           ...                                                ...
60635  TEST_60635                                                 -1
60636  TEST_60636  28640 13 28860 18 29082 20 29302 24 29524 27 2...
60637  TEST_60637  64 5 87 17 288 4 311 17 512 4 536 16 735 5 760...
60638  TEST_60638  14714 2 14936 5 15158 8 15382 8 15605 9 15830 ...
60639  TEST_60639  27028 5 27246 12 27465 17 27684 23 27904 27 28...

[60640 rows x 2 columns]>
